In [1]:
import torch
torch.__version__

'1.6.0'

In [2]:
torch.set_grad_enabled(True)

def scalars_to_tensor_bad(*a):
    """
    Will lose gradient :(
    """
    return torch.tensor([a], requires_grad=True)

def scalars_to_tensor_good(*a):
    """
    Will preserve gradient!
    """
    # https://discuss.pytorch.org/t/how-to-concatenate-to-a-tensor-with-a-0-dimension/6478
    # https://discuss.pytorch.org/t/what-is-the-best-way-to-append-scalar-to-tensor/54445/3
    a = [ai.unsqueeze(0) for ai in a]
    return torch.cat(a).squeeze(0)


def dfunc(x, scalars_to_tensor):
    print(f"scalars_to_tensor: {scalars_to_tensor}")
    assert x.requires_grad
    assert x.shape == (2,)
    # First, zero out gradients.
    if x.grad is not None:
        x.grad.zero_()

    y1, y2 = x + 2
    y1 *= 3
    y2 *= 5
    yh = scalars_to_tensor(y1, y2)
    print(y1, y2, yh)
    if not yh.requires_grad:
        print("Lost gradient :(")

    z = yh.mean()
    z.backward()
    print(x.grad)
    print()

In [3]:
x = torch.tensor([1., 2.], requires_grad=True)

dfunc(x, scalars_to_tensor_bad)
dfunc(x, scalars_to_tensor_good)

scalars_to_tensor: <function scalars_to_tensor_bad at 0x7fdc913f5e18>
tensor(9., grad_fn=<AsStridedBackward>) tensor(20., grad_fn=<AsStridedBackward>) tensor([[ 9., 20.]], requires_grad=True)
None

scalars_to_tensor: <function scalars_to_tensor_good at 0x7fdc913f5d08>
tensor(9., grad_fn=<AsStridedBackward>) tensor(20., grad_fn=<AsStridedBackward>) tensor([ 9., 20.], grad_fn=<SqueezeBackward1>)
tensor([1.5000, 2.5000])

